In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torchvision.transforms import transforms



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
df_train = pd.read_csv('/kaggle/input/video-games-rating-by-esrb/test_esrb.csv')
df_test = pd.read_csv('/kaggle/input/video-games-rating-by-esrb/Video_games_esrb_rating.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.shape

In [ ]:
sns.barplot(x=df_train['esrb_rating'].value_counts().index, y=df_train['esrb_rating'].value_counts());

In [ ]:
criterions = df_train.select_dtypes('int')

In [ ]:
criterion_dict = {}
for col in criterions:
    criterion_df = df_train.groupby('esrb_rating', as_index=False).agg({f'{col}': 'mean'})
    criterion_df[f'{col}'] = criterion_df[f'{col}'] * 100
    criterion_dict[f'{col}'] = criterion_df
    

In [ ]:
    criterion_dict

In [ ]:
dict_key = list(criterion_dict.keys())
fig, axs = plt.subplots(8, 4, figsize=(18,14))
fig.tight_layout()

for i, axis in enumerate(axs.reshape(32)):
    data = criterion_dict[dict_key[i]]
    axis.bar(x=data['esrb_rating'], height=data[f'{dict_key[i]}'])
    axis.title.set_text(dict_key[i])
    axis.set_ylabel('precentage')
    
plt.xlabel = 'Ratings'

As we can see that mature humour is not responsible for any class to be classify so we can drop this feature

In [ ]:
df_train.drop('mature_humor', axis=1,inplace=True)
df_test.drop('mature_humor', axis=1,inplace=True)

In [ ]:
target_encoder = LabelEncoder()
df_train['esrb_rating'] = target_encoder.fit_transform(df_train['esrb_rating'])
df_test['esrb_rating'] = target_encoder.transform(df_test['esrb_rating'])
df_train_clean = df_train.drop('title', axis=1)
df_test_clean = df_test.drop('title', axis=1)

In [ ]:
class GameDataset(Dataset):

    def __init__(self, df, target=None,transform=None):
        # Data Loading
        super(GameDataset, self).__init__()
        self.x = df.drop(target, axis=1).values
        self.y = df[target].values
#         self.x = torch.from_numpy(x)
#         self.y = torch.from_numpy(y)
        self.n_samples = self.x.shape[0]
        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.n_samples


dataset = GameDataset(df_train_clean, target='esrb_rating')
dataloader = DataLoader(dataset=dataset, batch_size=45, shuffle=True)

In [ ]:
df_train_clean

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(31, 128)
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(128, 64)
        self.l3 = torch.nn.Linear(64, 32)
        self.l4 = torch.nn.Linear(32, 4)
        
        
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.relu(self.l2(x))
        x = self.relu(self.l3(x))
        x = self.l4(x)
        return x

In [ ]:
num_epochs = 100
learning_rate = 0.001
model = Model()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(dataloader)

for epoch in range(num_epochs):
    for i, (datapoint, labels) in enumerate(dataloader):
        labels = labels.reshape(labels.shape[0],1)
        output = model(datapoint)
        loss = criterion(outputs, labels)

        optim.zero_grad()
        loss.backward()
        optim.step()

        if (i + 1) % 100 == 0:
            print(f"epochs {epoch + 1}/{num_epochs}, step {i + 1}/{n_total_steps}, loss = {loss.item():.4f}")